In [ ]:
pip install --upgrade pip && pip install -r requirements.txt

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score
import pickle
import boto.s3.connection
from boto.s3.key import Key
import os.path
from os import path

In [ ]:
# #############################################################################
# Initialize variables
# #############################################################################
input_file = "topics.csv"
output_file = "text-classifier.pkl"
nltk.download('stopwords')

In [ ]:
# #############################################################################
# S3 CONFIGURATION
# #############################################################################
default_access_key = 'xxxxx' # putting access key
default_secret_key = 'xxxxxx' # putting secret key
default_bucket_name = 'xxxxxxxx' # bucket name
default_host = 'xxxxxxx'

def saveFileToDisk(s3_host, s3_access_key, s3_secret_key, bucket_name, key_name,file_name):
    conn = boto.connect_s3 (
        aws_access_key_id = s3_access_key,
        aws_secret_access_key = s3_secret_key,
        host = s3_host,
        is_secure=False, 
        calling_format = boto.s3.connection.OrdinaryCallingFormat(),
    )
    bucket = conn.get_bucket(bucket_name)
    key = bucket.get_key(key_name)
    key.get_contents_to_filename(file_name)
    print(file_name + ' downloaded.')
    conn.close()

In [ ]:
# #############################################################################
# Load some categories from the training set
if not path.exists(input_file):
    saveFileToDisk(default_host,default_access_key,default_secret_key, default_bucket_name, input_file, input_file)

df = pd.read_csv(input_file)
original_headers = list(df.columns.values)

categories = df[original_headers[2]].unique().tolist()

print("Total dataset size: "+str(len(df)))

# Split data: 80% for training, 20% for testing
train, test = train_test_split(df, test_size=0.2)

print("Training dataset size: "+str(len(train)))
print("Test dataset size: "+str(len(test)))

train_x =  train[original_headers[1]].tolist()
train_y = train[original_headers[2]].tolist()

test_x = test[original_headers[1]].tolist()
test_y = test[original_headers[2]].tolist()

In [ ]:
# #############################################################################
# Extracting features from text files
# #############################################################################

# Tokenizing text
# https://stackabuse.com/text-classification-with-python-and-scikit-learn/
count_vect = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X_train_counts = count_vect.fit_transform(train_x)

In [ ]:
# #############################################################################
# Apply TFIF
# #############################################################################
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
# #############################################################################
# Create a simple naive bayes
# #############################################################################
clf = MultinomialNB().fit(X_train_tfidf, train_y)

In [ ]:
# #############################################################################
# Test simple naive bayes
# #############################################################################
X_test_counts = count_vect.transform(test_x)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
    
clf_predicted = clf.predict(X_test_tfidf)

print("Accuracy: {0:.2f} %".format(100 * accuracy_score(test_y, clf_predicted)))

In [ ]:
# #############################################################################
# Save model to a pickle file
# #############################################################################
with open(output_file, 'wb') as file:
    pickle.dump(clf, file)
print("Model file: "+output_file)